In [2]:
!pip install pyspark py4j


     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 57.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5e8cffcecb50e33e61499d8eb82bd39293342218a32c22c1eb244cb583534b7f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, array
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import functools
from pyspark.sql.functions import expr
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.types import DateType
from pyspark.sql import types as t
from pyspark.sql.types import *
import datetime 
from datetime import datetime

In [4]:
spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("OriginaisNetflix.parquet")
df.show()

+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|               Title|               Genre|         GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|     Length|MinLength|MaxLength|    Status|Active|Table|Language|         dt_inclusao|
+--------------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------+---------+---------+----------+------+-----+--------+--------------------+
|      House of Cards|     Political drama|     political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73| 42–59 min.|       42|       59|     Ended|     0|Drama| English|2021-03-16T21:20:...|
|       Hemlock Grove|     Horror/thriller|     horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33| 45–58 min.|       45|       58|

In [13]:
cols = ['Active', 'Genre']

df_first_step = df.select('*').withColumn("dt_inclusao_date",df['dt_inclusao'].cast(DateType()))\
                              .withColumn("Premiere_date",df['Premiere'].cast(TimestampType())).orderBy(cols, ascending= False )

# Não consegui transformar a coluna PREMIERE em formato de data, nenhuma conversão ela aceitava, trazia o erro "AttributeError: 'str' object has no attribute 'toordinal'",
# tentei de várias formas. Creio que seja porque ela é um número binário pelo que entendi.

df_second_step = df_first_step.withColumn('Seasons_format', regexp_replace('Seasons', 'TBA', 'a ser anunciado'))


df_third_step = df_second_step.withColumn('Data de Alteração', f.current_timestamp())

df_fourth_step = df_third_step.select(col('Title').alias('Título'),col('Genre').alias('Gênero'),col('GenreLabels').alias('Rótulos de Gênero')
                                      ,col('Premiere').alias('Pré Estreia'),col('Seasons_format').alias('Temporadas'),col('SeasonsParsed').alias('Temporadas Analisadas')
                                      ,col('EpisodesParsed').alias('Episódios Analisados'),col('Length').alias('Comprimento')
                                      ,col('MinLength').alias('Comprimento Mínimo'),col('MaxLength').alias('Comprimento Máximo'),col('Status')
                                      ,col('Active').alias('Ativo'),col('Table').alias('Tabela'),col('Language').alias('Linguagem'),col('dt_inclusao_date').alias('dt_inclusao'),col('Data de Alteração'))


df_fifth_step = df_fourth_step.select(col('Título'),col('Gênero'),col('Temporadas'),col('Pré Estreia'),col('Linguagem'),col('Ativo'),col('Status'),col('dt_inclusao'),col('Data de Alteração'))

df_fifth_step.show() 


df_fifth_step.write.option("header",True)\
        .option("delimiter",";")\
        .csv("/content/sample_data/teste.csv")

+--------------------+--------------------+--------------------+-----------+---------+-----+-------+-----------+--------------------+
|              Título|              Gênero|          Temporadas|Pré Estreia|Linguagem|Ativo| Status|dt_inclusao|   Data de Alteração|
+--------------------+--------------------+--------------------+-----------+---------+-----+-------+-----------+--------------------+
|          True Tunes|chrildrens musica...|     a ser anunciado|  12-Jul-19|  English|    1|Pending| 2021-03-16|2022-03-10 13:02:...|
|Super Monsters Mo...|chrildrens musica...|1 season, 4 episodes|  14-Sep-18|  English|    1|Pending| 2021-03-16|2022-03-10 13:02:...|
|Dance & Sing with...|chrildrens musica...|1 season, 11 epis...|  18-May-18|  English|    1|Pending| 2021-03-16|2022-03-10 13:02:...|
|      Super Monsters| childrens-animation|2 seasons, 16 epi...|  13-Oct-17|  English|    1|Renewed| 2021-03-16|2022-03-10 13:02:...|
|Charlie's Colorfo...| childrens-animation|1 season, 13 epis..